In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Holistic solution
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
left_knee_count = 0
right_knee_count = 0
left_knee_done = False
right_knee_done = False

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            
            left_knee_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y
            right_knee_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y

            if not right_knee_done and right_knee_y < 0.5:
                right_knee_count += 1
                if right_knee_count >= 5:
                    right_knee_done = True
                    cv2.putText(image, 'Good job! Right knee lift complete.', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.imshow('Knee Movement', image)
                    cv2.waitKey(2000)

            if right_knee_done and not left_knee_done and left_knee_y < 0.5:
                left_knee_count += 1
                if left_knee_count >= 5:
                    left_knee_done = True
                    cv2.putText(image, 'Good job! Left knee lift complete.', (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.imshow('Knee Movement', image)
                    cv2.waitKey(2000)
                    cap.release()
                    cv2.destroyAllWindows()
                    exit()

        cv2.imshow('Knee Movement', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
